In [1]:
import xlwings as xw
import pandas as pd

from Options_Math_Helpers import *
from Options_Math_Algebra import *
from Options_Math_Black_Scholes import *
from Options_SABR_Skew import *

oma  = OptionsMathAlgebra()
ombs = OptionsMathBlackScholes()
sabr = SABRSkew()

In [2]:
# Open workbook
wb = xw.Book('deribit.xlsx')

# Select sheet by name
sheet = wb.sheets['enrich']

# Read entire used range into a DataFrame
df = sheet.used_range.options(pd.DataFrame, header=True, index=False).value

In [3]:
cols = ['instrument_name', 
        'option_type', 
        'strike', 
        'expiration_time_nyc', 
        'years_to_expiry', 
        'underlying_price',
        'mark_price', 
        'best_bid_price', 
        'best_ask_price',
        'enriched_bid_iv', 
        'enriched_ask_iv',
        'enriched_mid_iv'
       ]

df = df.loc[:, [c for c in cols if c in df.columns]]

print('done')

done


In [4]:
df = df[df['enriched_mid_iv'].notna()]

In [5]:
call_mask = df['option_type'] == 'call'
date_mask = (df['years_to_expiry'] > 0.11) & (df['years_to_expiry'] < 0.12)

df = df.loc[call_mask & date_mask, :]

In [6]:
u = df['underlying_price']
k = df['strike']
t = df['years_to_expiry']
v = df['enriched_mid_iv'] / 100

In [7]:
df['vega'] = ombs.bs_vega(u=u, k=k, v=v, t=t) 

In [14]:
fwd_val = u.mean()

# MAYBE MAKE THIS BETTER
atm_vol = np.interp(fwd_val, k, v) 

print(fwd_val, atm_vol)

102261.12304347825 0.44097051470741594


In [25]:
beta = 0.5


#  MAKE THIS BETTER
initial_alpha = sabr.sabr_alpha(atm_vol, fwd_val, beta)
print(initial_alpha)

141.01484373599644


In [26]:
alpha, rho, nu, result = sabr.calibrate_sabr_weighted(fwd=u, strike=k, time=t, 
                                                      target_vols=v, 
                                                      beta=beta,
                                                      weighting='vega',
                                                      weights=df['vega'],
                                                      initial_guess=(144, 0.0, 0.5),
                                                      bounds=([1e-8, -0.999, 1e-8], [500.0, 0.999, 5.0]))
                                
print(alpha, beta, rho, nu)

135.55381963164533 0.5 -0.11208380086660506 1.9656966342353104


In [27]:
df['sabr_vol'] = sabr.sabr_vol(fwd=u, strike=k, time=t, 
                               alpha=alpha, beta=beta, rho=rho, nu=nu) 
                               
df['sabr_vol'] = df['sabr_vol'] * 100

In [28]:
# test if sabr_vol in between bid and ask?

In [29]:
sheet = wb.sheets('sabr1')
sheet.range('a1').options(index = False, header=1).value = df